# Extract from GCP

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade google-cloud-storage
!{sys.executable} -m pip install --upgrade gcsfs
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/willyraedy/Sync/SideProjects/RedditResearch/credentials-ebeb319739c4.json
import pandas as pd
import pickle

raw_df = pd.read_csv('gs://meli_thesis/All_Subreddits_With_Desc_And_Wikis', encoding='latin1')
df = raw_df.set_index('real_name')
with open('../data/allSubredditsEnriched.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

# Load Raw Dataframes

In [4]:
import pickle
import pandas as pd

with open('../data/allSubredditsEnriched.pickle','rb') as read_file:
    all_subreddits = pickle.load(read_file)

In [5]:
import pandas as pd

raw_excluded_subreddits = pd.read_csv('../data/manuallyExcluded.csv', encoding='latin1')

# Create List of Manually Excluded Reddits

In [6]:
excluded_list = raw_excluded_subreddits[raw_excluded_subreddits['Manually Exclude'] == 1].Subreddit.str[:-2]

# Create Network

In [2]:
import sys
import importlib
sys.path.append('../src/')

import reddit
reddit = importlib.reload(reddit)

import network
network = importlib.reload(network)

net = network.fetch_network(reddit_name='latinoamerica', dataframe=all_subreddits, max_depth=3, excluded_list=list(excluded_list))
df = pd.DataFrame(net)

# Convert Network to Edge List CSV Files

In [102]:
df.rename(columns={ 'parent': 'source', 'type_of_parent': 'source_type', 'reddit': 'target', 'type_of_reddit': 'target_type', 'type_of_edge': 'edge_type' }, inplace=True)
deduped = df.drop_duplicates(subset=['source', 'target', 'edge_type'])
deduped[['source', 'source_type', 'target', 'target_type', 'edge_type', 'context']].to_csv('../data/edgeList.csv')

In [110]:
unique_network_reddits = df.target.unique()
attribute_list = pd.DataFrame(unique_network_reddits)
attribute_list = attribute_list.merge(all_subreddits.reset_index(), how='left', left_on=0, right_on='real_name')
attribute_list[['real_name', 'desc', 'created_date', 'subs']].to_csv('../data/attributeList.csv')